In [33]:
# This is for memory handling 

In [34]:
from dotenv import load_dotenv
load_dotenv()
from langchain import hub
from langchain.agents import AgentExecutor, create_structured_chat_agent
from langchain.memory import ConversationBufferMemory
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage
from langchain_core.tools import Tool
from langchain.chat_models import ChatOpenAI
from langchain.agents.conversational.base import ConversationalAgent

In [35]:
def get_current_time(*args, **kwargs):
    """Returns the current time in H:MM AM/PM format."""
    import datetime

    now = datetime.datetime.now()
    return now.strftime("%I:%M %p")


def search_wikipedia(query):
    """Searches Wikipedia and returns the summary of the first result."""
    from wikipedia import summary

    try:
        # Limit to two sentences for brevity
        return summary(query, sentences=2)
    except:
        return "I couldn't find any information on that."

In [36]:
# Define the tools that the agent can use
tools = [
    Tool(
        name="Time",
        func=get_current_time,
        description="Useful for when you need to know the current time.",
    ),
    Tool(
        name="Wikipedia",
        func=search_wikipedia,
        description="Useful for when you need to know information about a topic.",
    ),
]

tools_name = ['Time','Wikipedia']

In [37]:
# prompt = hub.pull("hwchase17/structured-chat-agent")
# print(prompt)

prefix = "you are helpful assistant that will answer questions based on given input and tools you have."
suffix = """Begin!
Available tools:{tools}
Tool Names: {tool_names}
{chat_history}
Question: {input}
{agent_scratchpad}"""
format_instructions = """Use the following format:
"""
prompt = ConversationalAgent.create_prompt(
  tools,
  prefix=prefix,
  suffix=suffix,
  input_variables=["input", "agent_scratchpad", "chat_history","tool_names","tools"],
  format_instructions=format_instructions
)


In [38]:
# Initialize a ChatOpenAI model
llm = ChatOpenAI(model="gpt-4o")

In [39]:
# Create a structured Chat Agent with Conversation Buffer Memory
# ConversationBufferMemory stores the conversation history, allowing the agent to maintain context across interactions
memory = ConversationBufferMemory(
    memory_key="chat_history", return_messages=True)

In [40]:
# create_structured_chat_agent initializes a chat agent designed to interact using a structured prompt and tools
# It combines the language model (llm), tools, and prompt to create an interactive agent
agent = create_structured_chat_agent(llm=llm, tools=tools, prompt=prompt)

In [41]:
# AgentExecutor is responsible for managing the interaction between the user input, the agent, and the tools
# It also handles memory to ensure context is maintained throughout the conversation
agent_executor = AgentExecutor.from_agent_and_tools(
    agent=agent,
    tools=tools,
    verbose=True,
    memory=memory,  # Use the conversation memory to maintain context
    handle_parsing_errors=True,  # Handle any parsing errors gracefully
)

In [42]:
# Initial system message to set the context for the chat
# SystemMessage is used to define a message from the system to the agent, setting initial instructions or context
initial_message = "You are an AI assistant that can provide helpful answers using available tools.\nIf you are unable to answer, you can use the following tools: Time and Wikipedia."
memory.chat_memory.add_message(SystemMessage(content=initial_message))

In [43]:
# Chat Loop to interact with the user
while True:
    user_input = input("User: ")
    if user_input.lower() == "exit":
        break

    # Add the user's message to the conversation memory
    memory.chat_memory.add_message(HumanMessage(content=user_input))

    # Invoke the agent with the user input and the current chat history
    response = agent_executor.invoke({"input": user_input})
    print("Bot:", response["output"])

    # Add the agent's response to the conversation memory
    memory.chat_memory.add_message(AIMessage(content=response["output"]))



> Entering new AgentExecutor chain...
Could not parse LLM output: To answer your question about Barack Obama, I will use the Wikipedia tool to provide detailed information. Let's look up "Barack Obama" on Wikipedia. 

Wikipedia(query="Barack Obama")
For troubleshooting, visit: https://python.langchain.com/docs/troubleshooting/errors/OUTPUT_PARSING_FAILURE Invalid or incomplete responseCould not parse LLM output: I will use the Wikipedia tool to find information about Barack Obama.

Wikipedia(query="Barack Obama")
For troubleshooting, visit: https://python.langchain.com/docs/troubleshooting/errors/OUTPUT_PARSING_FAILURE Invalid or incomplete responseCould not parse LLM output: I will use the Wikipedia tool to find information about Barack Obama.

Wikipedia(query="Barack Obama")
For troubleshooting, visit: https://python.langchain.com/docs/troubleshooting/errors/OUTPUT_PARSING_FAILURE Invalid or incomplete responseCould not parse LLM output: It seems there is an issue with retrieving i

KeyboardInterrupt: 